In [1]:
import os
import re
import time
from contextlib import contextmanager
from operator import itemgetter

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import japanize_matplotlib
%matplotlib inline

import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import TfidfVectorizer as Tfidf
from sklearn.pipeline import make_pipeline, make_union, Pipeline
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import KFold, train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_log_error, r2_score

from google.cloud import storage

import re, MeCab
from glob import glob
import mojimoji
import warnings
warnings.simplefilter("ignore")

from keras.callbacks import EarlyStopping
from keras.layers.advanced_activations import PReLU
from keras.layers.core import Activation, Dense, Dropout
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential, load_model
from keras.utils import np_utils

#環境変数,
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../auth/My First Project.json"
bucket_name = "pj_horidasimono"
prefix="dataset/train/ElectricalAppliance"
#最大表示行数の指定（ここでは50行を指定）
pd.set_option('display.max_rows', 200)
#最大表示列数の指定（ここでは50列を指定）
pd.set_option('display.max_columns', 200)


In [2]:
def road_data_from_gcs(bucket_name, prefix):
    client = storage.Client()
    blobs = client.list_blobs(bucket_name, prefix=prefix)
    df = pd.DataFrame()
    for blob in blobs:
        bucket = client.get_bucket(bucket_name)
        r = storage.Blob(blob.name, bucket)
        content = r.download_as_string()
        df = df.append(pd.read_json(content))
        print(f"read file {blob.name}...")

    df = df.drop_duplicates(subset="url")
    df = df.reset_index(drop=True)
    return df

In [33]:
tagger = MeCab.Tagger("-Owakati")
def make_wakati(sentence):
    # MeCabで分かち書き
    sentence = tagger.parse(sentence)
    sentence = mojimoji.zen_to_han(sentence)
    # 半角全角英数字除去
    #sentence = re.sub(r'[0-9０-９a-zA-Zａ-ｚＡ-Ｚ]+', " ", sentence)
    # 記号もろもろ除去
    sentence = re.sub(r'[\．_－―─！＠＃＄％＾＆\-‐|\\＊\“（）＿■×+α※÷⇒—♬◉ᴗ͈ˬ●★☆⭐️⭕⚡⚠o①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬⑭⑮♡⭐︎〇◎◆♦▼◇△□(：〜～＋=)／*&^%$#@!~`)♪ᴖ◡ᴖ{}［］…\[\]\"\'\”\’:;<>?＜＞〔〕＼〈〉？、､。｡・,\./『』【】｢｣「」→←○《》≪≫\n\u3000⭕]+', "", sentence)
    # 絵文字除去
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           "]+", flags=re.UNICODE)
    sentence = emoji_pattern.sub(r'', sentence)
    # スペースで区切って形態素の配列へ
    #wakati = sentence.split(" ")
    # 空の要素は削除
    #wakati = list(filter(("").__ne__, wakati))
    return sentence

def title_torkenize(sentence):
    sentence = mojimoji.zen_to_han(sentence)
    sentence = re.sub("[\．_－―─！＠＃＄％＾＆\-‐|\\＊\“（）＿■×+α※÷⇒♬◉ᴗ͈ˬ—●★☆⭐️⭕⚡⚠o①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬⑭⑮♡⭐︎〇◎◆♦▼◇△□(：〜～＋=)／*&^%$#@!~`)♪ᴖ◡ᴖ{}［］…\[\]\"\'\”\’:;<>?＜＞〔〕＼〈〉？、､。｡・,\./『』【】｢｣「」→←○《》≪≫\n\u3000]", " ", sentence)
    sentence = re.sub("[あ-ん]", " ", sentence)
    sentence = re.sub("( |　)+", " ", sentence)
    sentence = sentence.lower()
    #〇〇様専用を除く
    sentence = re.sub("[^ ]*専用", "", sentence)
    sentence = re.sub("[^ ]*様", "", sentence)
    #1文字のアルファベットを除く
    sentence = re.sub(" [a-z]{1}[^(a-z)]", " ", sentence)
    # 絵文字除去
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           "]+", flags=re.UNICODE)
    sentence = emoji_pattern.sub(r'', sentence)
    sentence = sentence.strip()

    return sentence

def preprocess(df):
    df["price"] = df["price"].str.replace(r"\D", "").astype(np.float)
    
    #列ズレを修正
    pattern = re.compile(r"^(?!.*(傷や汚れあり|全体的に状態が悪い|やや傷や汚れあり|未使用に近い|目立った傷や汚れなし|新品、未使用)).+$")
    invalid = df["status"].str.match(pattern)

    df.loc[invalid, "description"] = df.loc[invalid, "description"] + "\n" + df.loc[invalid, "status"]
    df.loc[invalid, "status"]      = df.loc[invalid, "shipping"]
    df.loc[invalid, "shipping"]    = df.loc[invalid, "method"]
    df.loc[invalid, "method"]      = df.loc[invalid, "region"]
    df.loc[invalid, "period"]      = "未定"
    
    df["title"] = df["title"] + " " + df["sub_category_1"] + " " + df["sub_category_2"] + " " + df["brand"]
    #df["text"]  = df["title"] + " " + df["description"]

    df = df.drop(columns=["sub_category_1", "sub_category_2", "brand"])
    
    status_dict = {'新品、未使用': "best",
                   '未使用に近い': "Very Good",
                   '目立った傷や汚れなし': "good",
                   '傷や汚れあり': "Poor",
                   'やや傷や汚れあり': "very poor",
                   '全体的に状態が悪い': "worst"
                  }
    
    #配送負担をラベルエンコーディング
    shipping_dict = {'送料込み(出品者負担)': 0, '着払い(購入者負担)': 1}

    df["status"] = df["status"].map(status_dict)
    df["shipping"] = df["shipping"].map(shipping_dict)
    
    #トークナイズ
    df["title"] = df["title"].apply(title_torkenize)
    df["description"] = df["description"].apply(make_wakati)
    
    #不要列削除
    df = df.drop(columns=["url", "seller", "rating", "method", "region", "period", "recent_comment", "timestamp"])
    return df

In [7]:
from model_ridge import ModelRidge

In [28]:
df_ = road_data_from_gcs(bucket_name, prefix)

read file dataset/train/ElectricalAppliance20200921194130.json...
read file dataset/train/ElectricalAppliance20200921212935.json...
read file dataset/train/ElectricalAppliance20200921230505.json...
read file dataset/train/ElectricalAppliance20200922003032.json...
read file dataset/train/ElectricalAppliance20200922122051.json...
read file dataset/train/ElectricalAppliance20200922195104.json...
read file dataset/train/ElectricalAppliance20200923003024.json...
read file dataset/train/ElectricalAppliance20200923005724.json...
read file dataset/train/ElectricalAppliance20200923024628.json...
read file dataset/train/ElectricalAppliance20200923194105.json...
read file dataset/train/ElectricalAppliance20200923210407.json...
read file dataset/train/ElectricalAppliance20200923224903.json...
read file dataset/train/ElectricalAppliance20200924003019.json...
read file dataset/train/ElectricalAppliance20200924093732.json...
read file dataset/train/ElectricalAppliance20200924161706.json...
read file 

In [12]:
@contextmanager
def timer(name):
    t0 = time.time()
    yield
    print(f'[{name}] done in {time.time() - t0:.0f} s')
    
def on_field(f: str, *vec):
    return make_pipeline(FunctionTransformer(itemgetter(f), validate=False), *vec)

def to_records(df: pd.DataFrame):
    return df.to_dict(orient='records')

def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

vectorizer = make_union(
            on_field("title", Tfidf(max_features=100000, token_pattern="\w+")),
            on_field("description", Tfidf(max_features=100000, token_pattern="\w+", ngram_range=(1, 2))),
            on_field(['shipping', 'status'],
                 FunctionTransformer(to_records, validate=False), DictVectorizer()))

with timer("preprocess"):
  df = preprocess(df_.copy())
  X = df.drop(columns="price")
  y = df["price"]

  X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=0)

with timer("training"):
  model = ModelRidge(vectorizer)
  model.fit(X_train, y_train)
    
va_pred = np.expm1(model.predict(X_valid))
score = np.sqrt(mean_squared_log_error(y_valid, va_pred))
mape = np.mean(np.abs((y_valid - va_pred) / y_valid)) * 100
print("mape: {:.3f}%".format(mape))
print(score)

[preprocess] done in 67 s
[training] done in 113 s
mape: 37.187%
0.46028625975647264


## 前処理

In [47]:
def preprocess(df):
    df["price"] = df["price"].str.replace(r"\D", "").astype(np.float)
    
    #列ズレを修正
    pattern = re.compile(r"^(?!.*(傷や汚れあり|全体的に状態が悪い|やや傷や汚れあり|未使用に近い|目立った傷や汚れなし|新品、未使用)).+$")
    invalid = df["status"].str.match(pattern)

    df.loc[invalid, "description"] = df.loc[invalid, "description"] + "\n" + df.loc[invalid, "status"]
    df.loc[invalid, "status"]      = df.loc[invalid, "shipping"]
    df.loc[invalid, "shipping"]    = df.loc[invalid, "method"]
    df.loc[invalid, "method"]      = df.loc[invalid, "region"]
    df.loc[invalid, "period"]      = "未定"
    
    df["title"] = df["title"] + " " + df["sub_category_1"] + " " + df["sub_category_2"] + " " + df["brand"]
    df["text"]  = df["title"] + " " + df["description"]

    df = df.drop(columns=["sub_category_1", "sub_category_2", "brand", "description"])
    
    status_dict = {'新品、未使用': "best",
                   '未使用に近い': "Very Good",
                   '目立った傷や汚れなし': "good",
                   '傷や汚れあり': "Poor",
                   'やや傷や汚れあり': "very poor",
                   '全体的に状態が悪い': "worst"
                  }
    
    #配送負担をラベルエンコーディング
    shipping_dict = {'送料込み(出品者負担)': 0, '着払い(購入者負担)': 1}

    df["status"] = df["status"].map(status_dict)
    df["shipping"] = df["shipping"].map(shipping_dict)
    
    #不要列削除
    df = df.drop(columns=["url", "seller", "rating", "method", "region", "period", "recent_comment", "timestamp"])
    return df

In [16]:
df = preprocess(df_.copy())

In [168]:
df.head()

,title,price,status,shipping,description
0,ｼﾞｬﾝｸ iphone se silver 32 gb simﾌﾘｰ ｽﾏｰﾄﾌｫﾝ 携帯...,3000.0,Poor,0,iPhone SE Silver 32 GB SIM ﾌﾘｰ 最近 まで 使用 し て い ...
1,glidic tw 5000s ｵｰﾃﾞｨｵ機器 ｲﾔﾌｫﾝ,4000.0,best,0,ﾜｲﾔﾚｽ ｲﾔﾎﾝ です ｡ 新品 未 使用 です ｡ 1 度 も 開封 せ ず ､ 自宅...
2,zoom sj ｵｰﾃﾞｨｵ機器 他,5000.0,Very Good,0,ﾊﾝﾃﾞｨ ﾀｲﾌﾟ の ﾚｺｰﾀﾞ です 風防 付き な の で 屋外 で の 録音 に ...
3,canon ﾌﾟﾘﾝﾀｲﾝｸ純正品6色ｾｯﾄ 350 351 標準容量 pc ﾀﾌﾞﾚｯﾄ ...,2500.0,best,0,ﾌﾟﾘﾝﾀ 買い替え に 伴い 余剰 に なっ た の で 出品 し ます ｡ PGBK ...
4,panasonic np tcm4 2018年 生活家電 他,21000.0,good,0,ご覧 いただき ありがとう ござい ます ｡ Panasonic 食洗 機 NP TCM ...


In [29]:
df_description = preprocess(df_.copy())["description"]

In [30]:
for i, j in enumerate(df_description[0:10]):
    print(i, j)

0 iPhne SE Silver 32 GB SIM ﾌﾘｰ 最近 まで 使用 し て い た  iphneSE 第 1 世代 です  ﾀｯﾁ ﾊﾟﾈﾙ が たまに しか 反応 し ない ため  ｼﾞｬﾝｸ 品 と し て の 出品 です  本 品 は Sim ﾌﾘｰ 品 で  au  sftbank SIM で の 動作 は 確認 済み です  本体 以外 付属 品 は あり ませ ん  機種 名  iPhne SE ｶﾗｰ  Silver 容量  32 GB 購入 し た ｷｬﾘｱ  SIM ﾌﾘｰ 付属 品  なし 
1 ﾜｲﾔﾚｽ ｲﾔﾎﾝ です  新品 未 使用 です  1 度 も 開封 せ ず  自宅 保管 し おり まし た  使う 機会 が ない の で 出品 し ます  GLDIC ﾜｲﾔﾚｽ ｲﾔﾎﾝ Bluetth 
2 ﾊﾝﾃﾞｨ ﾀｲﾌﾟ の ﾚｺｰﾀﾞ です 風防 付き な の で 屋外 で の 録音 に 最適 です 数 回 しか 使っ て 無い の で ｷｽﾞ 等   全く あり ませ ん 動作 も 確認 済み です  問題 あり ませ ん 説明 書 は あり ませ ん が ﾈｯﾄ から DR 可 です 
3 ﾌﾟﾘﾝﾀ 買い替え に 伴い 余剰 に なっ た の で 出品 し ます  PGBK  BK  GY は ｾｯﾄ 品 を 購入 し て い た ため 先月 末 に 箱 は 開封 し て しまっ た の で  すぐ お 使い の 方 に お 譲り し たい と 思い ます  残り の 3 色 は  期限 まで いずれ も 18 か月 以上 あり ます   ｷﾔﾉﾝ  Cann 
4 ご覧 いただき ありがとう ござい ます  Panasnic 食洗 機 NP  TCM 4 知り合い に 譲り受け まし た が  私 は 使用 する 機会 が ない ため お 譲り いたし ます  2018 年 製造 品 に なり ます  使用 頻度 は 毎日 は 使わ ず 週 4 程度 です  通常 使用 に ともなう 軽微 な ｽﾚ ･ ｷｽﾞ あり ます よく 見 ない と 分から ない 程度 です  正面 向かっ て 左側 側面 に 小 傷 が あり ます が  小 傷 の ため あまり 気 に なり ませ ん  画像 10 枚 目 にて 

In [34]:
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

trainings = [TaggedDocument(words = data.split(),tags = [i]) for i,data in enumerate(df_description)]


In [38]:
m = Doc2Vec(documents= trainings, dm = 1, size=300, window=8, min_count=10, workers=4)

In [51]:
m.save('doc2vec.model')
#model = models.Doc2Vec.load('doc2vec.model')

In [47]:
m.infer_vector(["宜しく", "どうぞ", "付属", "品", "は", "画像", "が", "全て", "です",  "問題", "なく", "動作", "確認", "済み", "です"]).shape

(300,)

In [54]:
ls -lh

total 451M
drwxr-xr-x 2 root root 4.0K Oct 21 12:33 __pycache__/
-rw-r--r-- 1 root root  85M Oct 21 13:46 doc2vec.model
-rw-r--r-- 1 root root 183M Oct 21 13:46 doc2vec.model.docvecs.vectors_docs.npy
-rw-r--r-- 1 root root 183M Oct 21 13:46 doc2vec.model.docvecs.vectors_docs_norm.npy
-rw-r--r-- 1 root root 1.7K Oct 21 12:15 model_lgb.py
-rw-r--r-- 1 root root 3.9K Oct 21 12:31 model_nn.py
-rw-r--r-- 1 root root 2.0K Oct 21 12:31 model_ridge.py
-rw-r--r-- 1 root root 2.4K Oct 21 12:28 util.py
